## REACT multimodal multi agent with Pandas
#### by David O'Dell

A set of tools is defined whereby the model decides which is the most appropriate tool.  Tool choice is based on the user prompt and some guidance in the system prompt.  The tools include a simple python function (datetime), web search, and a Pandas query of synthetic patient data as typically found in a clinic.  The pandas query is a nested agent itself and is activated when the LLM understands that the question is related only to patient data.  The pandas supports text to graphic output by generating graphs based on the patient data.  


<img src="images/multi-agent-pandas.jpg" alt="Alternative text" />

## Installed libraries

In [ ]:
# ## langchain libraries
# %pip install -q langchain==0.3.24
# %pip install -q langchain-core==0.3.56
# %pip install -q langchain-community==0.3.23
# %pip install -q langchain-experimental==0.3.4
# %pip install -q langchain-openai==0.3.3
# %pip install -q langchain-text-splitters==0.3.8


# ## other libraries
# %pip install -q accelerate==1.3.0
# %pip install -q gradio==4.44.1
# %pip install -q matplotlib==3.10.0
# %pip install -q pydantic==2.10.6
# %pip install -q tabulate==0.9.0
# %pip install -q tavily-python==0.5.1
# %pip install -q transformers==4.48.3


In [ ]:
import torch
import sys
import os
import transformers
import pandas as pd
import gradio as gr
import matplotlib.pyplot as plt
import requests

from langchain.tools import tool
from langchain.agents import AgentType, initialize_agent, load_tools, AgentExecutor, create_react_agent
from langchain_core.prompts.prompt import PromptTemplate
from langchain_experimental.agents import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI
from langchain_community.tools import TavilySearchResults
from langchain_experimental.tools import PythonAstREPLTool
from pydantic import BaseModel

from datetime import datetime
from subprocess import call
from IPython.display import display
from PIL import Image

In [ ]:
# model_id = "meta/llama-3.1-70b-instruct"
# api_url = "http://192.168.51.9:30003/v1"


model_id = "meta/llama-3.1-70b-instruct"
api_url = "http://192.168.51.11:32001/v1"     ### WORKER NODE IP AND PORT FOR LLM

llm = ChatOpenAI(
    base_url=api_url,
    api_key="mykey1234",   ### this is a made up key, doesn't actually exist
    model=model_id,
    temperature=0,
    max_tokens=None,
)

In [ ]:
df = pd.read_csv('csv-files/healthcare.csv')
print(df.head(5))

In [ ]:
# from langchain.globals import set_verbose, set_debug
# set_debug(True)
# set_verbose(True)

### Web search API key

In [ ]:
os.environ["TAVILY_API_KEY"] = "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"


### Define tools

In [ ]:
@tool
def current_datetime(input: str) -> str:
    """Get the current date and time
    Returns:
        str: The current date and time
    """
    return datetime.now().strftime('%A %d %B %Y, %I:%M%p')


# @tool
# def python_tool(input: str) -> str:
#     """Perform python commands
#     Returns:
#         str: The result of the command
#     """    
#     # Execute the code
#     result = PythonAstREPLTool.invoke(input)
#     return {"result": result}


@tool
def web_search(input: str) -> str:
    """Perform a web search and return the results
    Returns:
        str: The search results
    """
    web_search_tool = TavilySearchResults(max_results=5) 
    result = web_search_tool.invoke(input)
    return {"result": result}


@tool
def pandas_agent(input: str) -> str:
    """Process the input using the pandas dataframe agent
    Returns:
        str: The result from the pandas dataframe agent
    """
    df_agent = create_pandas_dataframe_agent(
        llm,
        df,
        verbose=True,
        allow_dangerous_code=True,
        return_intermediate_steps=True, 
        agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    )

    result = df_agent.invoke(input)
    return {"result": result}

### Declare tools

In [ ]:
tools = [current_datetime, web_search, pandas_agent]

### Define system prompt

In [ ]:
template = """ 
You are a healthcare clinic assistant that has access to additional tools in order to answer the following questions as best you can
even if some questions are not exactly related to healthcare. 

You have access to the following tools:

{tools}

The web_search tool is good for searching the web for recent information about people, places and global data.

The current_datetime tool is great for telling the current time, referencing past and future times and calculating dates. 

The pandas_agent tool should only be used when being asked specific questions related to patient data.   
This tool allows you to access specific patient data in CSV format.
When using the pandas_agent tool, you must be careful in your final answer NOT to include any backticks as this will cause the LLM parsing to fail.
If you notice any backticks in the final answer, you must enclose them in brackets like this: [``] so that they do not cause parsing errors.


To use a tool, please use the following format:

'''
Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of these tools... [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat 3 times)
'''


When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:
'''
Thought: Do I need to use a tool? No
Final Answer: [your response here]
'''

Begin!

Question: {input}
Thought:{agent_scratchpad}
"""

### Integrate tools and Agent Executor

In [ ]:
prompt = PromptTemplate.from_template(template)
react_agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=react_agent, tools=tools, verbose=True, return_intermediate_steps=True)

### function used for Gradio GUI

In [ ]:
def handle_query(user_prompt):
    
    global plot_path
    plot_path = None  # Reset plot_path at the beginning

    query = agent_executor.invoke({"input": user_prompt})
    
    # Check if any figures are open
    if plt.get_fignums():
        # Save the plot if it exists
        plot_path = "images/my_chart.png"
        plt.savefig(plot_path)
        plt.close()

    if plot_path is None:
        return query["output"], None
    else:
        return query["output"], plot_path

In [ ]:
# response = handle_query("Create a barplot of the age distribution of the patients.")

# if plot_path:
#     print("\n\n======== PRINTING PLOT =========\n")
#     display(Image.open(plot_path))

In [ ]:
# response = handle_query("Create a pie chart plot of the number of procedures based on percentage.")

# if plot_path:
#     print("\n\n======== PLOT WAS CREATED, DISPLAYING PLOT =========\n")
#     display(Image.open(plot_path))

In [ ]:
# response = handle_query("Provide a list of the top 5 unique reasons for admissions in descending order..")

# if plot_path:
#     print("\n\n======== PLOT WAS CREATED, DISPLAYING PLOT =========\n")
#     display(Image.open(plot_path))

In [ ]:
# Create the Gradio interface with examples
iface = gr.Interface(
    fn=handle_query,
    inputs=gr.Textbox(lines=2, placeholder="Enter your query here..."),
    outputs=["text", "image"],
    title="Multi agent, multi tool chatbot (Web, Pandas and Datetime)",
    examples=[
        ["Create a colorful bar graph of the top 5 unique reasons for admissions in descending order."],
        ["What was the date of discharge for patients 33 and 40?  I would like to send them a small notice.  Can you provide me a small notice to them?"],
        ["Cross reference and plot the total charges of all patients with the reason for admission to determine which reason had the highest cost.   Please include the results from the cross reference and costs in your response formatted nicely."],
        ["Create a histogram of the age distribution of the patients under 30 years of age."],
        ["Create a pie chart plot of the number of procedures based on percentage."],
        ["What time would it be if it was two hours later from now?"],
        ["What is the plot to the movie Tron?"],
        ["Based on public information, what is the most recent population count of the state of Texas?"],
    ]
)

# Launch the interface
iface.launch(share=False, debug=True, server_name="192.168.51.11", server_port=7870, allowed_paths=["images/"])